In [1]:
# !pip install wget
# !sudo apt update
# !sudo apt install build-essential -y
# !sudo apt-get install manpages-dev -y
# !sudo apt-get update && apt-get install libgl1 -y
# !pip install opencv-python
# !pip install -U matplotlib
# !pip install tqdm
# !pip install pandas
# !pip install joblib
# !pip install scikit-learn
# !pip install lightgbm
# !pip install transformers
# !pip install --upgrade transformers
# !pip install sentencepiece
# !pip install protobuf==3.20.1
# !pip install text_unidecode
# !pip install seaborn
# !pip install pytorch_lightning
# !pip install pytorchvideo

In [28]:
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorchvideo.models import create_res_basic_head

import pytorch_lightning as pl
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler
from pytorchvideo.data.labeled_video_paths import LabeledVideoPaths
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
)

In [3]:
import shutil
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
tqdm.pandas()
import os.path as osp
import matplotlib.pyplot as plt
import wget
import zipfile
import seaborn as sn

In [4]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.12.1 True


## Datasets

In [5]:
%rm -rf /workspace/data/dev/
%rm -rf /workspace/data/val/

In [6]:
ann        = pd.read_csv('/workspace/data/yolo_bboxes.csv')
ann['fps'] = 32
ann.event  = ann.event.map({'play':0, 'challenge':1, 'throwin':2})
ann.drop_duplicates('event_id', inplace=True)

In [7]:
all_videos = os.listdir('/workspace/data/exp/')
np.random.seed(0)
np.random.shuffle(all_videos)
dev_vid, val_vid = all_videos[:3500], all_videos[3500:]

print(len(dev_vid), len(val_vid))

os.makedirs('/workspace/data/dev/')
os.makedirs('/workspace/data/val/')

for v in tqdm(dev_vid):
    shutil.copytree(f'/workspace/data/exp/{v}', f'/workspace/data/dev/{v}')
    
for v in tqdm(val_vid):
    shutil.copytree(f'/workspace/data/exp/{v}', f'/workspace/data/val/{v}')
    

100 100


100%|██████████| 100/100 [00:00<00:00, 305.05it/s]


In [8]:
from src.data import *
from src.model import *

In [9]:
dm = DFLDataModule_Slow(
    LabeledVideoPaths(
        [(f'/workspace/data/dev/{v}',
         ann.set_index('event_id').loc[dev_vid, 'event'].values[i]) for i,v in enumerate(dev_vid)]
    ),
    LabeledVideoPaths(
        [(f'/workspace/data/val/{v}',
         ann.set_index('event_id').loc[val_vid, 'event'].values[i]) for i,v in enumerate(val_vid)]
    ),
    clip_duration = 32/25,
    batch_size = 4
)

## Training

In [ ]:
# csn_r101 32  infer 3.5 it/s
# csn_r101 16
# slow_r50     infer 4 it/s
# r2plus1d_r50 infer 3 it/s
# x3d_l        bug
# slowfast_r50 bug

In [46]:
model = torch.hub.load("facebookresearch/pytorchvideo:main", "slow_r50", pretrained=True)
model.blocks[:-1].requires_grad_(False)
model.blocks[-1] = create_res_basic_head(in_features=2048, out_features=dm.num_labels)


# model = torch.hub.load("facebookresearch/pytorchvideo:main", "csn_r101", pretrained=True)
# #model.blocks[:-1].requires_grad_(False)
# model.blocks[-1] = create_res_basic_head(in_features=model.blocks[-1].proj.in_features, out_features=dm.num_labels)


# model = torch.hub.load("facebookresearch/pytorchvideo:main", "r2plus1d_r50", pretrained=True)
# #model.blocks[:-1].requires_grad_(False)
# model.blocks[-1] = create_res_basic_head(in_features=model.blocks[-1].proj.in_features, out_features=dm.num_labels)


# model = torch.hub.load("facebookresearch/pytorchvideo:main", "slowfast_r50", pretrained=True)
# model.blocks[:-1].requires_grad_(False)
# model.blocks[-1] = create_res_basic_head(in_features=model.blocks[-1].proj.in_features, out_features=dm.num_labels)


# model = torch.hub.load("facebookresearch/pytorchvideo:main", "x3d_l", pretrained=True)
# #model.blocks[:-1].requires_grad_(False)
# model.blocks[-1] = create_res_basic_head(in_features=model.blocks[-1].proj.in_features, out_features=dm.num_labels)


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/X3D_L.pyth" to /root/.cache/torch/hub/checkpoints/X3D_L.pyth


  0%|          | 0.00/47.7M [00:00<?, ?B/s]

In [47]:
%rm -rf lightning_logs

In [48]:
chkallback = ModelCheckpoint(monitor='val_loss', dirpath='./lightning_logs/exp/', filename='best_model')
classifier = Classifier(model, lr=1e-4)
trainer    = pl.Trainer(gpus=1, precision=16, max_epochs=4,
                        logger=CSVLogger("lightning_logs", name="exp"),
                        callbacks=[chkallback])
trainer.fit(classifier, dm)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | Net              | 4.4 M 
1 | loss_fn | CrossEntropyLoss | 0     
2 | val_acc | Accuracy         | 0     
---------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
8.743     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x192 and 2048x3)

In [37]:
pd.read_csv('./lightning_logs/exp/version_0/metrics.csv', sep=',').bfill().drop_duplicates('epoch')

,val_loss,val_acc,epoch,step,train_loss
0,1.173611,0.75,0,24,0.018572
2,1.176149,0.81,1,49,0.050422
4,0.834779,0.79,2,74,0.046122
6,0.834945,0.81,3,99,0.076502


In [39]:
%ls ./lightning_logs/exp/

'epoch=2-step=75.ckpt'   version_0/
